In [4]:
import powerplantmatching as pm
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
import numpy as np
import datetime
from scipy.stats import expon
from scipy.optimize import curve_fit
from astropy.visualization import hist
#for fitting:
from scipy.stats import expon, rv_discrete
from scipy.optimize import curve_fit
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#for plotting Markov Chain graph
import networkx as nx
import matplotlib.pyplot as plt


#helper functions

def import_df(path, cols = ['StartTS', 'EndTS', 'TimeZone', 'Status', 'Type', 'AreaCode',
       'AreaTypeCode', 'AreaName', 'MapCode', 'PowerResourceEIC', 'UnitName',
       'ProductionType', 'InstalledCapacity', 'AvailableCapacity',
       'Reason']):
    """
    imports and preprocess data_frame
    path: string containing path of csv file containing table
    cols: list of column names to select in df
    returns: non redundat dataframe with only failures
    """
    
    df = pd.read_csv(path, sep = "\t", parse_dates = [0,1])
    df = df[cols] #selects only column
    df = df.drop_duplicates(subset = ["UnitName","StartTS"]) #deletes redundant rows
    #df = df[(df["Reason"] == "Failure")] # WHERE | (df["Reason"] == 'Foreseen Maintenance')
    #maybe can do df[df["Reason"] in reasons]?
    return df

def get_week(date):
    """
    input: date in date_time format
    output: what week of the year the date corresponds to
    """
    return date.week


def truncate(number, digits) -> float:
    # Improve accuracy with floating point operations, to avoid truncate(16.4, 2) = 16.39 or truncate(-1.13, 2) = -1.12
    nbDecimals = len(str(number).split('.')[1]) 
    if nbDecimals <= digits:
        return number
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

def markov_graph(transitions, seed = 42, digits = 4, title = ""):
    """
    input: transitions, a dictionary having as 
    keys: touples with 2 elements being the from state and from state
    values: the transition probability
    output: markov chain graph
    """
    G = nx.MultiDiGraph()

    for transition, probability in transitions.items():
        state_from, state_to = transition
        if probability != 0: 
        #if probability state_from to state_to is not 0 we add an edge to the graph
            G.add_edge(state_from, state_to, weight=truncate(probability, digits))

    #create positions of nodes: dictionary with coordinates
    pos = nx.spring_layout(G, seed) 

    # Increase the scale to avoid overlap
    pos = {k: [v[0] * 2, v[1] * 2] for k, v in pos.items()}

    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=8, font_color='black',
            connectionstyle='arc3,rad=0.1')

    # Annotate edges manually with adjusted positions to avoid overlap
    for edge, weight in labels.items():
        (x, y) = pos[edge[0]]
        text_x = 3/4*x + 1/4*pos[edge[1]][0]
        text_y = 3/4*y + 1/4*pos[edge[1]][1]
        #shift text to avoid overlap
        text_y += 0.2 if edge[0] == edge[1] else 0


        plt.text(text_x, text_y, f"{weight}", fontsize=8, color='blue', verticalalignment='center',
                 horizontalalignment='center')
    plt.title(title)
    plt.show()
    
    
def combine_overlaps(df):
    """
    this functions combines any time overlaps present in the dataframe for each generator
    so that for every time t there is at most one row describing the generator at time t.
    df: dataframe containing UnitName, StartTS, EndTS
    """

    # Step 1: Sort the DataFrame
    df.sort_values(by=["UnitName", "StartTS"], inplace=True)

    # Step 2 and 3: Combine overlapping intervals
    result = []
    current_interval = None
    n_rows = df.shape[0]
    perc = n_rows // 100 *  5

    for k, row in df.iterrows():
        if k % perc == 0:
            print(f"percentage of rows parsed = {k / n_rows *100:.2f}%")
        if current_interval is None:
             current_interval = row.copy()
        elif row["StartTS"] >= current_interval["EndTS"] or row["UnitName"] != current_interval["UnitName"]:
            # No overlap or new UnitID
            result.append(current_interval)
            current_interval = row.copy()
        else:
            # Overlapping intervals, update the EndTS
            current_interval["EndTS"] = row["StartTS"]

    result_df = pd.DataFrame(result)

    return result_df


def get_markov_probs(df, states_column):
    """
    input:
    df: dataframe having as columns: states_column, "ProductionType", "StartTS", "UpTime"
    states_column: string with name of column where the state of the generator is saved
    output: dictionary having as keys tuples with two states and the associated probability transition
    """
    states = list(df[states_column].unique())
    states.append("Running")
    transitions = []
    for x in states:
        for y in states:
            transitions.append((x,y))
            
    transitions_counter = dict(zip(transitions, [0]*len(transitions)))
    GenGroups = df.groupby("UnitName")
    previous_state = "Running"
    current_state = "Running"
    for unit_name, unit_df in GenGroups:
        unit_df = unit_df.sort_values(["StartTS"])
        #count transition occurante for unit
        for index, row in unit_df.iterrows():
            uptime = row["RunningTime"]
            #get current state from row
            current_state = row[states_column]

            if pd.isna(uptime):
                #if uptime == "Nan" then it was the first recorded instance of the generator in the dataframe so before it was running.
                previous_state = "Running"
            elif uptime > 10 / (60 * 24): # and previous_state != "Running"
                #if the generator had some time between the previous row than the previous state was running
                #and we must add 1 to previousprevious state and running
                transitions_counter[(previous_state, "Running")] += 1
                previous_state = "Running"    

            transitions_counter[(previous_state, current_state)] += 1
            #the current state becomes the previous_state
            previous_state = current_state

    #get the transtions probabilities
    transitions_probs = transitions_counter
    counter_dict = dict(zip(states, [0]*len(states)))
    for state in states:
        for transition, counter in transitions_probs.items():
            if transition[0] == state:
                counter_dict[state] += counter 

    for transition, counter in transitions_probs.items():
        if counter_dict[transition[0]] != 0:
            #if transition[0] occurs at least one time
            transitions_probs[transition] = transitions_probs[transition] / counter_dict[transition[0]]
    return transitions_probs

def weighted_values(values, probabilities, size):
    bins = np.add.accumulate(probabilities)
    return values[np.digitize(np.random.random_sample(size), bins)]

def next_state_markov(markov, current_state):
    possible_states = []
    transition_probs = []
    for key, prob in markov.items():
        if key[0] == current_state and prob != 0:
            possible_states.append(key[1])
            transition_probs.append(prob)
    #if len(possible_states) == 0:
    #    return current_state
    #else:
    return weighted_values(np.array(possible_states), np.array(transition_probs),1)[0]

def get_gen_type(geo_df):
    geo_to_entsoe_gen_d = {
        "Hard Coal": "Fossil Hard coal ", 
        'Lignite' : 'Fossil Brown coal/Lignite ',
        'Oil': "Fossil Oil ",
        'Waste': "Waste ",
        'Natural Gas': "Fossil Gas ",#o ci andrebbe qualcos altro
        #'Hydro',
        'Nuclear': "Nuclear ", 
        'Other' : "Other ", 
        'Solar':"Solar ",
        'Wind':"Wind Onshore ",
        'Geothermal':"Geothermal "
        }
    #given a unit row of geo dataframe give gen type in ENTSOE format
    #in geo there is just one windpower type (not onshore or offshore) or maybe you can see from the dataframe
    fuel_type = geo_df["Fueltype"]
    gen_type = geo_df["Technology"]
    if fuel_type == "Hydro":
        if gen_type == "Reservoir":
            return 'Hydro Water Reservoir '
        elif gen_type == "Run-Of-River":
            return 'Hydro Run-of-river and poundage '
        elif gen_type == "Pumped Storage":
            return 'Hydro Pumped Storage '
    elif fuel_type in geo_to_entsoe_gen_d.keys():
        return geo_to_entsoe_gen_d[fuel_type]
    else:
        print(f"Generetor {gen_type},{fuel_type} not found classfied as Other")
        return "Other "



In [5]:
#helper functions
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

# Questions:
1. There is no onshore / offshore windpower ---> remove them
2. What is Hydro without further specification 
3. For what timeframe should I generate data?
4. Solar ----> remove them

# TODO:
- ~remove redundant gens in geo~
- ~add seed for random variables~

In [24]:
#Set parameters
#import generators
geo = pm.data.GEO()
geo = geo.groupby("projectID").head(1)

#set start time and end time for which to generate scenarios
start_time = np.datetime64("2023-01-01T00:00:00")
end_time = np.datetime64("2023-06-01T00:00:00")

#sed seed for random generation for replicable scenario generation
seed = 2193 #set 

#import distribution parameters
statetime_df = pd.read_csv("exponential_statetime_df.csv")
capacity_df = pd.read_csv("kernel_capacity_df.csv")
markov_d = np.load("markov_state_change_d.npy", allow_pickle = True).item()

## Dataset descripiton

    - projectID - Immutable identifier of the power plant
        
    - Power plant name - claim of each database

    - Fueltype - {Bioenergy, Geothermal, Hard Coal, Hydro, Lignite, Nuclear, Natural Gas, Oil, Solar, Wind, Other}

    - Technology - {CCGT, OCGT, Steam Turbine, Combustion Engine, Run-Of-River, Pumped Storage, Reservoir}

    - Set - {Power Plant (PP), Combined Heat and Power (CHP), Storages (Stores)}

    - Capacity - [MW]

    - Duration - Maximum state of charge capacity in terms of hours at full output capacity

    - Dam Information - Dam volume [Mm^3] and Dam Height [m]

    - Geo-position - Latitude, Longitude

    - Country - EU-27 + CH + NO (+ UK) minus Cyprus and Malta

    - YearCommissioned - Commmisioning year of the powerplant

    - RetroFit - Year of last retrofit




In [5]:
geo_gen_types = geo["Technology"].unique()
geo_fuel_types = geo["Fueltype"].unique()
#grouped  = df.groupby(["UnitName"]).first().reset_index()
#entsoe_gen_types = list(grouped["ProductionType"].unique())

In [26]:
capacity_df

,Unnamed: 0,ProductionType,Type,p.u.
0,0,Fossil Hard coal,Forced,0.673105
1,1,Fossil Hard coal,Forced,0.643840
2,2,Fossil Hard coal,Forced,0.000000
3,4,Fossil Hard coal,Forced,0.614574
4,6,Fossil Hard coal,Forced,0.702371
...,...,...,...,...
454222,573766,Fossil Gas,Forced,-0.030303
454223,573767,Fossil Gas,Planned,0.000000
454224,573769,Fossil Gas,Planned,0.000000
454225,573771,Fossil Gas,Planned,0.000000


In [28]:
# Recreate Capacity density kernels

grouped_df = capacity_df.groupby(["ProductionType", "Type"])
capacity_d = {}
#idea: calculate prob for capacity = 0, remove and the use non parametric fit.
for production_type, prod_df in grouped_df:
    #calculate per unit available capacity
    PU = prod_df["p.u."]
    p_0 = np.sum(PU == 0) / len(PU)
    PUplus = np.sort(PU[PU != 0]) #non zero capacities
    if len(PUplus) != 0:
        
        colors = ["r"]
        kernels = ["gaussian"]
        lw = 2
        #Uncomment to plot
        
        #plt.figure()
        for color, kernel in zip(colors, kernels):
            kde = KernelDensity(kernel=kernel, bandwidth=0.05).fit(PUplus[:, np.newaxis])
            capacity_d[production_type] = (p_0, kde)
            log_dens = kde.score_samples(PUplus[:, np.newaxis]) * len(PUplus) / len(PU) #scale to be prob on non zero values
            Y = np.exp(log_dens)
            #plt.plot(PUplus, Y, color = color, label = kernel)
            
        """
        # Plot histogram

        plt.xlim(0, 1)
        
        plt.hist(PU, bins=min(500, int(np.ceil(len(PU) ** (1. / 3))*3/2)) + 3 , edgecolor='black', alpha=0.7, density = True)

        # Customize the plot (add labels, title, etc.)
        plt.xlabel('p.u. Available Capacity')
        plt.ylabel('Frequency')
        plt.title(f'{production_type}')
        plt.legend()
        plt.show()
        """


In [30]:
# Generating Scenarios

#Starting state --> we roll random running time for every generator the end
# todo: make the markov chaing run for a while to make the various generators get further in states
#np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
np.random.seed(seed)
#grouped  = df.groupby(["UnitName"]).first().reset_index()
gen_names = list(geo["projectID"])
gen_types = []
for index, geo_row in geo.iterrows():
    gen_type = get_gen_type(geo_row)
    if gen_type is None:
        tech = geo_row["Technology"]
        fuel = geo_row["Fueltype"]
        print("Weird thing:", tech, fuel)
        gen_types.append("Other ")
    else:
        gen_types.append(gen_type)

n_gens = len(gen_names)
pre_run_hours = 6*30*24 #time the chain is run for before producing data

states = ["Running", "Forced", "Planned"]
state_df = pd.DataFrame({"UnitName": gen_names, "ProductionType": gen_types, "State":["Running"]*n_gens, "Counter":[0]*n_gens, "Capacity":[1]*n_gens})
perc = np.ceil(pre_run_hours / 100)

for h in np.arange(pre_run_hours):
    
    if h % (perc) == 0:
        print(f"{h/pre_run_hours *100} %")
        print(state_df.head(5))
    
    for index, gen_row in state_df[state_df["Counter"] == 0].iterrows():
        gen_name = gen_row["UnitName"] #remove?
        gen_type = gen_row["ProductionType"]
        current_state = gen_row["State"]
        markov = markov_d[gen_type] #get associate markov chain
        
        new_state = next_state_markov(markov, current_state) #get new state of the generator
        
        scale = statetime_df.loc[statetime_df["ProductionType"] == gen_type, new_state + "Time"]
        new_counter = np.ceil(np.random.exponential(scale, 1))[0] #get number of hours spent in new_state
        
        #get the capacity of the generator in the currnent state
        if new_state == "Running":
            new_capacity = 1
        elif (gen_type, new_state) in capacity_d.keys():
            p_zero, pu_pdf = capacity_d[(gen_type, new_state)]
            if np.random.random_sample(1)[0] <= p_zero:
                new_capacity = 0
            else:
                new_capacity = pu_pdf.sample(1)[0][0]
                if new_capacity < 0:
                    new_capacity = 0
                elif new_capacity > 1:
                    new_capacity = 1
        else:
            #todo this should not happen
            #print(f"no capacity distribution for {gen_type}")
            new_capacity = 1
        
        state_df.loc[index, ["State", "Counter", "Capacity"]] = [new_state, new_counter, new_capacity]
        
    state_df.loc[state_df["Counter"] != 0, "Counter"] -= 1
    
#idea: 
# while t < 10 years roll dices take final state and final counter

Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing: nan Hydro
Weird thing

38.7037037037037 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      228  1.000000
1  GEO-45150  Fossil Hard coal   Running      123  1.000000
2  GEO-45719  Fossil Hard coal   Planned       23  0.749148
3  GEO-41956  Fossil Hard coal   Planned        5  0.968042
4  GEO-41974  Fossil Hard coal   Running       75  1.000000
39.72222222222222 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      184       1.0
1  GEO-45150  Fossil Hard coal   Running       79       1.0
2  GEO-45719  Fossil Hard coal   Running      409       1.0
3  GEO-41956  Fossil Hard coal   Running      105       1.0
4  GEO-41974  Fossil Hard coal   Running       31       1.0
40.74074074074074 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      140       1.0
1  GEO-45150  Fossil Hard coal   Running       35       1.0
2  GEO-45719  Fossil Hard coal   Running 

62.12962962962963 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      536       1.0
1  GEO-45150  Fossil Hard coal   Running      337       1.0
2  GEO-45719  Fossil Hard coal   Running      184       1.0
3  GEO-41956  Fossil Hard coal   Running       46       1.0
4  GEO-41974  Fossil Hard coal   Running      252       1.0
63.14814814814815 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      492       1.0
1  GEO-45150  Fossil Hard coal   Running      293       1.0
2  GEO-45719  Fossil Hard coal   Running      140       1.0
3  GEO-41956  Fossil Hard coal   Running        2       1.0
4  GEO-41974  Fossil Hard coal   Running      208       1.0
64.16666666666667 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      448       1.0
1  GEO-45150  Fossil Hard coal   Running      249       1.0
2  GEO-45719  Fossil Hard coal   Running

84.53703703703704 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running        1       1.0
1  GEO-45150  Fossil Hard coal   Running      313       1.0
2  GEO-45719  Fossil Hard coal   Running      137       1.0
3  GEO-41956  Fossil Hard coal   Running      183       1.0
4  GEO-41974  Fossil Hard coal   Running      650       1.0
85.55555555555556 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running       16       1.0
1  GEO-45150  Fossil Hard coal   Running      269       1.0
2  GEO-45719  Fossil Hard coal   Running       93       1.0
3  GEO-41956  Fossil Hard coal   Running      139       1.0
4  GEO-41974  Fossil Hard coal   Running      606       1.0
86.57407407407408 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Planned       12       0.0
1  GEO-45150  Fossil Hard coal   Running      225       1.0
2  GEO-45719  Fossil Hard coal   Running

In [31]:
# Generating Scenarios
# given a dataframe having UnitName and UnitType create scenarios for each UnitName
# 

np.random.seed(seed)
tot_hours = (end_time - start_time) / np.timedelta64(1, "h")
data_list = []
current_time = start_time
#state_df = pd.DataFrame({"UnitName": gen_names, "ProductionType": gen_types, "State":["Running"]*n_gens, "Counter":[0]*n_gens, "Capacity":[1]*n_gens})

perc = np.ceil(tot_hours / 100)
columns = ["TimeStamp"] + gen_names

for h in np.arange(tot_hours):
    
    if h % (10*perc) == 0:
        print(f"{h/tot_hours *100} %")
        print(state_df.head(5))
    
    for index, gen_row in state_df[state_df["Counter"] == 0].iterrows():
        gen_name = gen_row["UnitName"] #remove?
        gen_type = gen_row["ProductionType"]
        current_state = gen_row["State"]
        markov = markov_d[gen_type] #get associate markov chain
        
        new_state = next_state_markov(markov, current_state) #get new state of the generator
        
        scale = statetime_df.loc[statetime_df["ProductionType"] == gen_type, new_state + "Time"]
        new_counter = np.ceil(np.random.exponential(scale, 1))[0] #get number of hours spent in new_state
        
        #get the capacity of the generator in the currnent state
        if new_state == "Running":
            new_capacity = 1
        elif (gen_type, new_state) in capacity_d.keys():
            p_zero, pu_pdf = capacity_d[(gen_type, new_state)]
            if np.random.random_sample(1)[0] <= p_zero:
                new_capacity = 0
            else:
                new_capacity = pu_pdf.sample(1,random_state = seed)[0][0]
                if new_capacity < 0:
                    new_capacity = 0
                elif new_capacity > 1:
                    new_capacity = 1
        else:
            #todo this should not happen
            #print(f"no capacity distribution for {gen_type}")
            new_capacity = 1
        
        state_df.loc[index, ["State", "Counter", "Capacity"]] = [new_state, new_counter, new_capacity]
        
    state_df.loc[state_df["Counter"] != 0, "Counter"] -= 1
    current_time = current_time + np.timedelta64(1, "h") #move forward one hour
    new_row = [current_time] + list(state_df["Capacity"])
    row_d = dict(zip(columns, new_row))
    data_list.append(row_d)

scenario = pd.DataFrame(data_list)
   

0.0 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      558  1.000000
1  GEO-45150  Fossil Hard coal   Running      227  1.000000
2  GEO-45719  Fossil Hard coal   Running      657  1.000000
3  GEO-41956  Fossil Hard coal   Running       29  1.000000
4  GEO-41974  Fossil Hard coal   Planned       33  0.248235
10.20971302428256 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      188       1.0
1  GEO-45150  Fossil Hard coal   Running      331       1.0
2  GEO-45719  Fossil Hard coal   Running      287       1.0
3  GEO-41956  Fossil Hard coal   Running       55       1.0
4  GEO-41974  Fossil Hard coal   Running      237       1.0
20.41942604856512 %
    UnitName     ProductionType    State  Counter  Capacity
0  GEO-45151  Fossil Hard coal   Running      281  1.000000
1  GEO-45150  Fossil Hard coal    Forced       21  0.948149
2  GEO-45719  Fossil Hard coal   Planned       20  1.0